In [1]:
pip install ecpy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycryptodome

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sympy

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install requests

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import math
import time
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from Crypto.Hash import SHA3_256, HMAC, SHA256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import random
import re
import json

from random import randint, seed
from ecpy.curves import Curve
from Crypto.Hash import SHA3_256
from Crypto import Random   # a bit better secure random number generation 
import math

In [5]:
E = Curve.get_curve('secp256k1')
n = E.order
p = E.field
P = E.generator
a = E.a
b = E.b
print("Base point:\n", P)
print("p :", p)
print("a :", a)
print("b :", b)
print("n :", n)

k = Random.new().read(int(math.log(n,2)))
k = int.from_bytes(k, byteorder='big')%n

Q = k*P
print("\nQ:\n", Q)
print("Q on curve?", E.is_on_curve(Q))

Base point:
 (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798 , 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
p : 115792089237316195423570985008687907853269984665640564039457584007908834671663
a : 0
b : 7
n : 115792089237316195423570985008687907852837564279074904382605163141518161494337

Q:
 (0xf478225f2ca46fcdfef20b3ff3937a7c817c9ef450514ab7a1732ade631aa6cb , 0x36a53d1ff5c8855749a34e8a6dfa29880e5b0a9e458d4e6768a257f5d2ef4468)
Q on curve? True


In [7]:
API_URL = 'http://10.92.52.175:5000/'

stuID =  25139  ## Change this to your ID number

#server's Identitiy public key
IKey_Ser = Point(93223115898197558905062012489877327981787036929201444813217704012422483432813 , 8985629203225767185464920094198364255740987346743912071843303975587695337619, E)

#Send Public Identitiy Key Coordinates and corresponding signature
def IKRegReq(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'IKPUB.X': x, 'IKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegReq"), json = mes)		
    if((response.ok) == False): print(response.json())

#Send the verification code
def IKRegVerify(code):
    mes = {'ID':stuID, 'CODE': code}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "IKRegVerif"), json = mes)
    if((response.ok) == False): raise Exception(response.json())
    print(response.json())

#Send SPK Coordinates and corresponding signature
def SPKReg(h,s,x,y):
    mes = {'ID':stuID, 'H': h, 'S': s, 'SPKPUB.X': x, 'SPKPUB.Y': y}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "SPKReg"), json = mes)		
    if((response.ok) == False): 
        print(response.json())
    else: 
        res = response.json()
        return res['SPKPUB.X'], res['SPKPUB.Y'], res['H'], res['S']

#Send OTK Coordinates and corresponding hmac
def OTKReg(keyID,x,y,hmac):
    mes = {'ID':stuID, 'KEYID': keyID, 'OTKI.X': x, 'OTKI.Y': y, 'HMACI': hmac}
    print("Sending message is: ", mes)
    response = requests.put('{}/{}'.format(API_URL, "OTKReg"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Send the reset code to delete your Identitiy Key
#Reset Code is sent when you first registered
def ResetIK(rcode):
    mes = {'ID':stuID, 'RCODE': rcode}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetIK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Sign your ID  number and send the signature to delete your SPK
def ResetSPK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetSPK"), json = mes)		
    print(response.json())
    if((response.ok) == False): return False
    else: return True

#Send the reset code to delete your Identitiy Key
def ResetOTK(h,s):
    mes = {'ID':stuID, 'H': h, 'S': s}
    print("Sending message is: ", mes)
    response = requests.delete('{}/{}'.format(API_URL, "ResetOTK"), json = mes)		
    if((response.ok) == False): print(response.json())



In [8]:
# IK Generation
sA = random.randint(1, n-2) # random secret key
sA = 53233605298669497722986980302469268232522496292809231123336496649869308259002
Q = sA * P  # generating public key
Q_x = Q.x
Q_y = Q.y
print("sA:", sA)
print("Q:", Q)
print("Q.x:", Q_x)
print("Q.y:", Q_y)

sA: 53233605298669497722986980302469268232522496292809231123336496649869308259002
Q: (0x14213fe9fb406ae3de7ce604bdc984a6441ef5cc5f2b57d33a9c96bff265e644 , 0x1abb5e94c8eff0999235235e1f3edffe53d12727946ee0c1a4f7b13c45693e9)
Q.x: 9105004042950320749475578173702996851762995069655837995435802152573491078724
Q.y: 755699202479272054922048722208735631432459381462857736356380499077106144233


In [9]:
# Signature Generation for IK
k = random.randint(1, n-2)
R = k * P
r = R.x % n

r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
m_byte_array = stuID.to_bytes((stuID.bit_length() +7)//8, byteorder = 'big')
r_m = r_byte_array + m_byte_array #concatenation of r and m where m is stuID
print(r_byte_array)
print(m_byte_array)
print(r_m)


h = SHA3_256.new(r_m)
h = int.from_bytes(h.digest(), byteorder='big') % n


s = (k - (sA * h)) % n
print("s:", s)
print("h:", h)

b'\xe2\xeb\x83\xea\xed\xdb\xa3X\x93JN\xe2\xc2\xa9\xf8z\xcc#\x05\xcc2\x91\xd8\xbcf}5\xf9\xeb\xf8\xf3\xce'
b'b3'
b'\xe2\xeb\x83\xea\xed\xdb\xa3X\x93JN\xe2\xc2\xa9\xf8z\xcc#\x05\xcc2\x91\xd8\xbcf}5\xf9\xeb\xf8\xf3\xceb3'
s: 19677716633679573044967781310320752831995692486129512728926291644874365753732
h: 11119324924536009736665777875826339055445614518519737802721208624385571883604


In [32]:
IKRegReq(h, s, Q_x, Q_y)

Sending message is:  {'ID': 25139, 'H': 71229627804431217312713731644888871768719827141354613904065035040829530777224, 'S': 52090617265907631408467595702876736489699008798207174714049253900559611060218, 'IKPUB.X': 9105004042950320749475578173702996851762995069655837995435802152573491078724, 'IKPUB.Y': 755699202479272054922048722208735631432459381462857736356380499077106144233}


In [33]:
CODE = 383087
IKRegVerify(CODE)

Sending message is:  {'ID': 25139, 'CODE': 383087}
Registered successfully


In [34]:
RCODE = 283463

In [10]:
# SPK Generation
spk_private = random.randint(1, n-2) # random secret key
spk_private = 68806696133384947541911215590293679233538678492859848006909180781567185984645
spk_pub = spk_private * Q  # generating public key
spk_pub_x = spk_pub.x
spk_pub_y = spk_pub.y
print("spk_private:", spk_private)
print("spk_pub:", spk_pub)
print("spk_pub_x:", spk_pub_x)
print("spk_pub_y:", spk_pub_y)

# Signature Generation for SPK

# calculating to be signed message
spk_pub_x_byte_array = spk_pub_x.to_bytes((spk_pub_x.bit_length() +7)//8, byteorder = 'big')
spk_pub_y_byte_array = spk_pub_y.to_bytes((spk_pub_y.bit_length() +7)//8, byteorder = 'big')
spk_x_y = spk_pub_x_byte_array + spk_pub_y_byte_array #concatenation of spk.x and spk.y 
m = spk_x_y
#print("spk_pub_x_byte_array", spk_pub_x_byte_array)
#print("spk_pub_y_byte_array", spk_pub_y_byte_array)
#print("concatenation of spk.x and spk.y:", spk_x_y)

# signature generation
k = random.randint(1, n-2)

R = k * P #P # Q is public IK
r = R.x % n
r_byte_array = r.to_bytes((r.bit_length() +7)//8, byteorder = 'big')
r_m = r_byte_array + m # concatenation of r and m where m is concatenation of spk.x and spk.y 


h = SHA3_256.new(r_m)
h = int.from_bytes(h.digest(), byteorder='big') % n

s = (k - (sA * h)) % n

print("h:", h)
print("s:", s)

spk_private: 68806696133384947541911215590293679233538678492859848006909180781567185984645
spk_pub: (0xc9ccfdc9f55be2e9544ca510e824e3dfdb7091e1a22ec72b95adeb6a6690b309 , 0x9f77187e6b35750676b6eb10dddd25354669b2c44b8e7baee5dcf1fa72c24dd2)
spk_pub_x: 91277070953065663896295258784311555083273663796409260265740237570154192483081
spk_pub_y: 72128166775603992717335251772452157822193335145469096027365938003749086842322
h: 68975290966468059351190438777310957537904953271245692071820834840045490692956
s: 97097110769674868447341168419991388088047837688022658476112728070414488896876


In [36]:
SPKReg(h, s, spk_pub_x, spk_pub_y)

Sending message is:  {'ID': 25139, 'H': 99050794899465936661610030166643218732709063479765855172743455036518385083664, 'S': 44069841590814678657653897474087527163525949711625715486439247339443688774720, 'SPKPUB.X': 91277070953065663896295258784311555083273663796409260265740237570154192483081, 'SPKPUB.Y': 72128166775603992717335251772452157822193335145469096027365938003749086842322}


(85040781858568445399879179922879835942032506645887434621361669108644661638219,
 46354559534391251764410704735456214670494836161052287022185178295305851364841,
 85044830889796253571729350975661241042992375374441662462539708185257601377291,
 7613849236463760334895211055274054255938630463536025794347835572470601614597)

In [11]:
server_SPK_pub = (85040781858568445399879179922879835942032506645887434621361669108644661638219,
 46354559534391251764410704735456214670494836161052287022185178295305851364841,
 85044830889796253571729350975661241042992375374441662462539708185257601377291,
 7613849236463760334895211055274054255938630463536025794347835572470601614597)

server_H = server_SPK_pub[2]
server_S = server_SPK_pub[3]
server_SPK_pub_x = server_SPK_pub[0]
server_SPK_pub_y = server_SPK_pub[1]
Q = sA * P  


#Signature Verification:
server_spk_pub_x_byte_array = server_SPK_pub_x.to_bytes((server_SPK_pub_x.bit_length() +7)//8, byteorder = 'big')
server_spk_pub_y_byte_array = server_SPK_pub_y.to_bytes((server_SPK_pub_y.bit_length() +7)//8, byteorder = 'big')
server_spk_x_y = server_spk_pub_x_byte_array + server_spk_pub_y_byte_array # concatenation of server_spk.x and server_spk.y 

V = (server_S*P) + (server_H*IKey_Ser) 
v = V.x % n
v_byte_array = v.to_bytes((v.bit_length() +7)//8, byteorder = 'big')
v_m = v_byte_array + server_spk_x_y
h_ = SHA3_256.new(v_m)
h_ = int.from_bytes(h_.digest(), byteorder='big') % n

print("h_:", h_)
print("h :", server_H)
if(server_H == h_):
    print("verified")
else:
    print("not verified")


h_: 85044830889796253571729350975661241042992375374441662462539708185257601377291
h : 85044830889796253571729350975661241042992375374441662462539708185257601377291
verified


In [23]:
# Generating HMAC Key (KHMAC):
server_SPK_pub_points = Point(server_SPK_pub_x, server_SPK_pub_y, E)
T = spk_private * server_SPK_pub_points
T_x = T.x
T_y = T.y

T_x_byte_array = T_x.to_bytes((T_x.bit_length() +7)//8, byteorder = 'big')
T_y_byte_array = T_y.to_bytes((T_y.bit_length() +7)//8, byteorder = 'big')
joker = bytes("NoNeedToRideAndHide", 'utf-8')
U = T_x_byte_array + T_y_byte_array + joker

K_HMAC = SHA3_256.new(U)

print("T_x_byte_array:", T_x_byte_array)
print("T_y_byte_array:", T_y_byte_array)
print("U:", U)
print("K_HMAC:", K_HMAC.digest())

T_x_byte_array: b'\x0f\xb1\x17+>\xf6\xb8\xd2\xca5`C\xf0\xe85=\xd6\xd50SgPU\xbb\x07\x9a_\xb5\xbf&\x16W'
T_y_byte_array: b'\'_\xc3p\xbb"\xba\x95\x0e\t\xcb\xc6=\xc0\xc5\x16\x88A_\x9b\x89M\xc7\x05\xf5\xba\x9b\xf5<\xcb\xa2\xca'
U: b'\x0f\xb1\x17+>\xf6\xb8\xd2\xca5`C\xf0\xe85=\xd6\xd50SgPU\xbb\x07\x9a_\xb5\xbf&\x16W\'_\xc3p\xbb"\xba\x95\x0e\t\xcb\xc6=\xc0\xc5\x16\x88A_\x9b\x89M\xc7\x05\xf5\xba\x9b\xf5<\xcb\xa2\xcaNoNeedToRideAndHide'
K_HMAC: b'*%R0k\xa8\x1a\xee\xc5q\x1eC\x8eu\xa3\x073>d\xe6\xbc\x8b\xb9\x1a\nE\x1b\xf7x\xbc\x1dc'


In [24]:
# OTKs generation
hmac_array = [] 
OTK_public = []
OTK_private = []

for i in range(10):
    print("for ID:", i)
    otk_private = random.randint(1, n-2) # random secret key
    otk_pub = otk_private * P # generating public key

    otk_pub_x = otk_pub.x
    otk_pub_y = otk_pub.y

    print("private:", otk_private)
    print("public:", otk_pub)
    print("public_x:", otk_pub_x)
    print("public_y:", otk_pub_y)

    otk_pub_x_byte_array = otk_pub_x.to_bytes((otk_pub_x.bit_length() +7)//8, byteorder = 'big')
    otk_pub_y_byte_array = otk_pub_y.to_bytes((otk_pub_y.bit_length() +7)//8, byteorder = 'big')
    otk_public_x_y = otk_pub_x_byte_array + otk_pub_y_byte_array #concatenation of x and y 

    hmac = HMAC.new(key = K_HMAC.digest(), msg = otk_public_x_y, digestmod = SHA256)
    
    print("hmac.hexdigest():", hmac.hexdigest())
    hmac_array.append(hmac)
    OTK_public.append(otk_pub)
    OTK_private.append(otk_private)

    OTKReg(i, otk_pub_x, otk_pub_y, hmac.hexdigest())
    print('------------------------------------------')


for ID: 0
private: 61830615680376783745858199796063924219722469598859981131360422531702033867739
public: (0x316e3d476e12c12ff9ec0bdeda9fde71a4f8420fdc01fd0786777eb93151461c , 0x6ade61170c3cff355c1433bd0c248b7d758b40985e62a72b18c01f0d39a0d937)
public_x: 22358105689981110192877136013451989349486874741082270169087518311817025766940
public_y: 48338072088970290690419900236245617258263392210805815947298339506119138924855
hmac.hexdigest(): 127ad144adcb8e2888fca6824891ee4c4be609f4870850707383383ca551fd81
Sending message is:  {'ID': 25139, 'KEYID': 0, 'OTKI.X': 22358105689981110192877136013451989349486874741082270169087518311817025766940, 'OTKI.Y': 48338072088970290690419900236245617258263392210805815947298339506119138924855, 'HMACI': '127ad144adcb8e2888fca6824891ee4c4be609f4870850707383383ca551fd81'}
The server couldn't verify the hmac!!
------------------------------------------
for ID: 1
private: 113222668204580479107246076442462548307149633188995441190312141663974698376195
public: (0x826cbd